In [48]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd 

#load the dataset 
data=gutenberg.raw('shakespeare-hamlet.txt')

#save to the file 

with open('text_file.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\amani\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [49]:
#data preprocessing 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

with open('text_file.txt','r') as file:
    data=file.read().lower()

#tokenizer-creating indexes for the words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
total_words=len(tokenizer.word_index)+1
total_words


4818

In [50]:
#create input sequences

input_seq=[]

for line in data.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_seq=token_list[:i+1]
        input_seq.append(n_gram_seq)

In [51]:
input_seq

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [52]:
#padding sequences
max_len=max([len(x) for x in input_seq])



In [53]:
input_seq=np.array(pad_sequences(input_seq,maxlen=max_len,padding='pre'))

In [54]:
input_seq

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [55]:
import tensorflow as tf
x,y=input_seq[:,:-1],input_seq[:,-1]


In [56]:
#converting into categories for y
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [58]:
#training model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

#define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

#compiling the model

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_4 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_2 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               100400    
                                                                 
 dense_2 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [63]:
#training model

history= model.fit( x_train, y_train,
    epochs=120,
    validation_data=(x_test, y_test),
    verbose=1)


Epoch 1/120
644/644 [==============================] - 16s 25ms/step - loss: 6.3967 - accuracy: 0.0408 - val_loss: 7.0543 - val_accuracy: 0.0443
Epoch 2/120
644/644 [==============================] - 16s 24ms/step - loss: 6.1543 - accuracy: 0.0498 - val_loss: 7.0956 - val_accuracy: 0.0482
Epoch 3/120
644/644 [==============================] - 15s 24ms/step - loss: 6.0132 - accuracy: 0.0526 - val_loss: 7.1536 - val_accuracy: 0.0519
Epoch 4/120
644/644 [==============================] - 15s 24ms/step - loss: 5.8864 - accuracy: 0.0575 - val_loss: 7.2460 - val_accuracy: 0.0565
Epoch 5/120
644/644 [==============================] - 15s 24ms/step - loss: 5.7681 - accuracy: 0.0599 - val_loss: 7.2875 - val_accuracy: 0.0571
Epoch 6/120
644/644 [==============================] - 15s 24ms/step - loss: 5.6528 - accuracy: 0.0652 - val_loss: 7.4108 - val_accuracy: 0.0587
Epoch 7/120
644/644 [==============================] - 15s 24ms/step - loss: 5.5325 - accuracy: 0.0717 - val_loss: 7.4835 - val_ac

In [64]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list =tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>= max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):] #ensure the sequence length matches max_sequence length
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return none


In [67]:
input_text="You are very good"
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len=max_sequence_len)
print(next_word)

father


In [71]:
#save the model
import pickle
model.save('next_word_lstm.keras')

#saving tokenizer
with open('tokenizer.pkl','wb') as file:
    pickle.dump(tokenizer,file,protocol=pickle.HIGHEST_PROTOCOL)